# Introduction to Few-Shot Learning and Meta Learning

Link: https://www.borealisai.com/en/blog/tutorial-2-few-shot-learning-and-meta-learning-i/

![metalearning](https://www.borealisai.com/media/filer_public_thumbnails/filer_public/a9/d6/a9d67824-b556-45f3-929a-290dbec5b44f/1_metalearningi_1.png__3000x1372_q85_subsampling-2.png)

Fig. 1  Meta-learning framework. An algorithm is trained using a series of training tasks. Here, each task is a 3-way-2-shot classification problem because each training task contains a support set with three different classes and two examples of each. During training the cost function assesses performance on the query set for each task in turn given the respective support set. At test time, we use a completely different set of tasks, and evaluate performance on the query set, given the support set. Note that there is no overlap between the classes in the two training tasks {cat, lamb, pig}, {dog, shark, lion} and between those in the test task {duck, dolphin, hen}, so the algorithm must learn to classify image classes in general rather than any particular set.

## Introduction

Humans can recognize new object classes from very few instances. However, most machine learning techniques require thousands of examples to achieve similar performance. 

The goal of few-shot learning is to classify new data having seen only a few training examples. In the extreme, there might only be a single example of each class (one shot learning). In practice, few-shot learning is useful when training examples are hard to find (e.g., cases of a rare disease), or where the cost of labelling data is high.

Few-shot learning is usually studied using **N-way-K-shot classification**. Here, we aim to discriminate between 
$N$ classes with $K$ examples of each. A typical problem size might be to discriminate between $N=10$ classes with only $K=5$ samples from each to train from. 

We cannot train a classifier using conventional methods here; any modern classification algorithm will depend on far more parameters than there are training examples, and will generalize poorly.

If the data is insufficient to constrain the problem, then one possible solution is to gain experience from other similar problems. To this end, most approaches characterize few-shot learning as a meta-learning problem.

## The meta learning framework

In the classical learning framework, we learn a how to classify from training data and evaluate the results using test data. 

In the meta-learning framework, we learn how to **learn** to classify given a set of **training tasks** and evaluate using a set of **test tasks** (figure 1); In other words, we use one set of classification problems to help solve other unrelated sets.

Here, each task mimics the few-shot scenario, so for **N-way-K-shot classification**, each task includes $N$ classes with $K$ examples of each. 

These are known as the **support set** for the task and are used for learning how to solve this task. In addition, there are further examples of the same classes, known as a **query set**, which are used to evaluating the performance on this task. 

Each task can be completely **non-overlapping**; we may never see the classes from one task in any of the others. The idea is that the system repeatedly sees instances (tasks) during training that match the structure of the final few-shot task, but contain different classes.

At each step of meta-learning, we update the model parameters based on a randomly selected training task. The **loss function** is determined by the classification performance on the query set of this training task, based on knowledge gained from its support set. Since the network is presented with a different task at each time step, it must learn how to discriminate data classes in general, rather than a particular subset of classes.

To evaluate few-shot performance, we use a set of test tasks. Each contains only unseen classes that were not in any of the training tasks. For each, we measure performance on the query set based on knowledge of their support set.

## Approaches to meta-learning

Approaches to meta-learning are diverse and there is no consensus on the best approach. However, there are three distinct families, each of which exploits a different type of prior knowledge:

- **Prior knowledge about similarity:** We learn embeddings in training tasks that tend to separate different classes even when they are unseen.

- **Prior knowledge about learning:** We use prior knowledge to constrain the learning algorithm to choose parameters that generalize well from few examples.

- **Prior knowledge of data:** We exploit prior knowledge about the structure and variability of the data and this allows us to learn viable models from few examples.

An overview these methods can be seen in figure 2. In this review, we will consider each family of methods in turn. 

![few_shot_learning_methods](https://www.borealisai.com/media/filer_public_thumbnails/filer_public/84/f1/84f1c168-adbe-4cac-9c10-451143eb4bea/meta_learning_i_table_fig_2.png__3000x1625_q85_subsampling-2.png)

Figure 2. Few-shot learning methods can be divided into three families. The first family learns prior knowledge about the similarity and dissimilarity of classes (in the form of embeddings) from training tasks. The second family exploits prior knowledge about how to learn that it has garnered from training tasks. The third family exploits prior knowledge about the data and its likely variation that is has learned from training tasks.


## Prior knowledge of similarity

This family of algorithms aims to learn compact representations (embeddings) in which the data vector is mostly unaffected by intra-class variations but retains information about class membership. Early work focused on pairwise comparators which aim to judge whether two data examples are from the same or different classes, even though the system may not have seen these classes before. Subsequent research focused on multi-class comparators which allow assignment of new examples to one of several classes.


## Pairwise comparators

Pairwise comparators take two examples and classify them as either belonging to the same or different classes. This differs from the standard N-way-K-shot configuration and does not obviously map onto the above description of meta-learning although as we will see later there is in fact a close relationship.

### Siamese networks

[Koch et al. (2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf) trained a model that outputs the probability $P_r(y_a=y_b)$ that two data examples $x_a$ and $x_b$ belong to the same class (Figure 3a). The two examples are passed through identical multi-layer neural networks (hence Siamese) to create two embeddings. The component-wise absolute distance between the embeddings is computed and passed to a subsequent comparison network that reduces this distance vector to a single number. This is passed though a sigmoidal output for classification as being the same or different with a cross-entropy loss.

![siamese_nn](https://www.borealisai.com/media/filer_public_thumbnails/filer_public/86/06/8606c20f-ac08-488a-a49a-0e8edc6fd390/metalearningi-2.png__3000x1185_q85_subsampling-2.png)

Figure 3. Pairwise comparators. a) **Siamese networks** take two examples $x_a$ and $x_b$ and return the probability $P_r(y_a=y_b)$ that they are the same class. They do this by passing each example through an identical network (hence Siamese) and then using the pairwise difference between the embeddings as the basis of the decision. b) **Triplet networks** take two examples of the same class $x_a$ and $x_+$ and one of a different class $x_−$ and pass all three through identical networks to create three embeddings. The triplet loss encourages the embeddings of examples from the same class to be closer together than those from different classes. c) In the test phase for triplet networks, we pass two examples $x_a$ and $x_b$ through the same network and judge whether they come from the same class or not based on the distance.

During training, each pair of examples are randomly drawn from a super-set of training classes. Hence, the system learns to discriminate between classes is **general**, rather than two classes in particular. In testing, completely different classes are used. Although this does not have the formal structure of the N-way-K-shot task, the spirit is similar.

### Triplet networks

Triplet networks ([Hoffer & Ailon 2015](https://arxiv.org/abs/1412.6622)) consist of three identical networks that are trained by triplets ${x_+,x_a,x_−}$ of the form (positive, anchor, negative). The positive and anchor samples are from the same class, whereas the negative sample is from a different class. The learning criterion is triplet loss which encourages the anchor to be closer to the positive example than it is to the negative example in the embedding space (Figure 3b). Hence it is based on two pairwise comparisons.

After training, the system can take two examples and establish whether they are from the same or different classes, by thresholding the distance in the learned embedding space. This was employed in the context of face verification by [Schroff et al. (2015)](https://arxiv.org/abs/1503.03832). This line of work is part of a greater literature on learning distance metrics (see [Suarez et al. 2018](https://arxiv.org/abs/1812.05944) for overview).

## Multi-class comparators

Pairwise comparators can be adapted to the N-way-K-shot setting by assigning the class for an example in the query set based on its maximum similarity to one of the examples in the support set. However, multi-class comparators attempt to do the same thing in a more principled way; here the representation and final classification are learned in an end-to-end fashion.

In this section, we'll use the notation $x_{nk}$ to denote the 
$k$th support example from the $n$th class in the N-Way-K-Shot classification task, and $y_{nk}$ to denote the corresponding label. 

For simplicity, we'll assume there is a single query example $\hat{x}$ and the goal is to predict the associated label $\hat{y}$.

### Matching Networks

Matching networks ([Vinyals et al. 2016](https://arxiv.org/abs/1606.04080)) predict the one-hot encoded query-set label $\hat{y}$ as a weighted sum of all of the one-hot encoded support-set labels ${y_nk}^{N,K}_{n,k=1}$. The weight is based on a computed similarity $a[\hat{x},x_{nk}]$ between the query-set data $\hat{x}$ and each training example 
${x_nk}^{N,K}_{n,k=1}$.